# Unsupervised Representation Learning With Deep Convolutional Generative Adversial Networks
* [paper](https://arxiv.org/pdf/1511.06434v2.pdf)

In [43]:
import torch
import torch.nn as nn

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Network

In [42]:
from typing import List

# experimented with how i created network to have more flexibility based on 
# what i've seen with other code

class DownsampleBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int=5, 
                 stride: int=2, padding: int=2):
        super().__init__()
        self.use_batchnorm = True
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride,
                              padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(.2)

    def forward(self, X):
        out = self.conv(X)
        if self.use_batchnorm:
            out = self.bn(out)
        out = self.act(out)
        return out

class UpsampleBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, 
                 kernel_size: int=5, stride: int=2, padding: int=2,
                 out_padding: int=1):
        super().__init__()
        self.use_batchnorm = True
        self.convT = nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
                                        stride, padding, out_padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU()

    def forward(self, X):
        out = self.convT(X)
        if self.use_batchnorm:
            out = self.bn(out)
        out = self.act(out)
        return out

class Generator(nn.Module):
    def __init__(self, channel_sizes: List[int], z_dim: int=100,
                 img_size: int=64):
        super().__init__()
        num_samples = len(channel_sizes)-1
        # dimensions of input volume for upsampling network
        # every time downsample, img_size halves
        self.in_dim = img_size//(2**num_samples)
        # depth of input volume for upsamling network
        self.in_channels = channel_sizes[0]
        # size of input volume to upsampling network flattend
        flat_size = channel_sizes[0] * self.in_dim * self.in_dim
        up_layers = []
        for i in range(num_samples):
            up_layers.append(UpsampleBlock(channel_sizes[i],
                                           channel_sizes[i+1]))
        self.img_size = img_size
        # final layer no batchnorm and use Tanh activation
        up_layers[-1].use_batchnorm = False
        up_layers[-1].act = nn.Tanh()
        self.upsample = nn.Sequential(*up_layers)
        self.fc = nn.Sequential(
            nn.Linear(z_dim, flat_size),
            nn.ReLU()
        )

    def forward(self, z: torch.Tensor):
        out = self.fc(z)
        out = out.view(-1, self.in_channels, self.in_dim, self.in_dim)
        out = self.upsample(out)
        return out

class Discriminator(nn.Module):
    def __init__(self, channel_sizes: List[int], img_size: int=64):
        super().__init__()
        num_samples = len(channel_sizes)-1
        # dimensions of output volume from downsampling network
        # everytime downsample, img_size halves
        out_dim = img_size//(2**num_samples)
        # depth of output volume from downsampling network
        out_channels = channel_sizes[-1]
        # size of output volume from downsampling network flattend
        flat_size = out_channels * out_dim * out_dim
        down_layers = []
        for i in range(num_samples):
            down_layers.append(DownsampleBlock(channel_sizes[i],
                                               channel_sizes[i+1]))
        # no batchnorm on input layer
        down_layers[0].use_batchnorm = False
        self.downsample = nn.Sequential(*down_layers)
        self.fc = nn.Sequential(
            nn.Linear(flat_size, 1),
            nn.Sigmoid()
        )

    def forward(self, X: torch.Tensor):
        out = self.downsample(X)
        out = torch.flatten(out, 1, 3)
        out = self.fc(out)
        return out